In [1]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 150
rcParams["figure.dpi"] = 150
import numpy as np
import matplotlib.pylab as plt

In [5]:
# Working in SI units, because I'm an idiot.
kmps = 1.e3 # km/s in m/s
Vsquared = 220. * 220. * kmps * kmps # (220 km/s)^2 in (m / s)^2
def potential(position, amplitude=Vsquared):
    assert position.shape == (3, )
    r = np.sum(position * position)
    return amplitude * np.log(r)
def acceleration(position, amplitude=Vsquared):
    assert position.shape == (3, )
    r = np.sum(position * position)
    return -1. * amplitude * position / (r * r)

In [6]:
# Some time and length conventions
yr = 3.154e7 # 1 yr in s
tmax = 1.e9 * yr # 1 Gyr in s ; total integration time per orbit
deltat = 1.e7 * yr # 0.01 Gyr in s ; default integration step size
pc = 3.0857e16 # 1 pc in m

In [7]:
velocity = 100. * kmps * np.random.normal(size=3)
position = 100. * pc * np.random.normal(size=3)
print(position, velocity)

[ -4.44459786e+17  -2.52627062e+18   4.91882512e+18] [ 143877.26337901 -107201.98145796  -91567.93321185]


In [ ]:
def integrate(x, v, amplitude=Vsquared, stepsize=deltat, duration=tmax):
    xn = 1. * x
    vnhalf = 1. * v + 0.5 * stepsize * acceleration(x, amplitude=amplitude)
    nsteps = int(tmax / deltat)
    xs = np.zeros(nsteps)
    ys = np.zeros(nsteps)
    for n in range(nsteps):
        xs[n] = xn
        vs[n] = whatever!

In [ ]:
xs, vs = integrate(position, velocity)